In [1]:
%%capture
! pip install trl
! pip install datasets
! pip install rdkit
! pip install peft
! pip install git+https://github.com/bitsandbytes-foundation/bitsandbytes.git

In [3]:
from datasets import load_dataset , Dataset
from trl import SFTConfig, SFTTrainer
import pandas as pd

## building sft dataset manual

In [55]:
df = pd.read_csv("/content/drive/MyDrive/Papers/Original_Papers/Llm_drug_prediction/Code/reasoning_data_api/test_reason.csv")
dset = Dataset.from_pandas(df)

In [56]:
df_sample = df.head()

In [57]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw, AllChem, Lipinski, Crippen, rdMolDescriptors
from rdkit.Chem import rdDepictor
import pandas as pd

def get_molecule_properties(smiles):
    """
    Compute molecular properties and analyses for a given SMILES string.

    Args:
        smiles (str): SMILES string of the molecule.

    Returns:
        dict: A dictionary containing all computed properties and analyses.
    """
    # Initialize the molecule
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        raise ValueError("Invalid SMILES string provided.")

    # Dictionary to store all properties
    properties = {}


    properties["Molecular_Weight"] = Descriptors.MolWt(molecule)
    properties["LogP"] = Crippen.MolLogP(molecule)
    properties["Molecular_Refractivity"] = Crippen.MolMR(molecule)
    properties["TPSA"] = Descriptors.TPSA(molecule)
    properties["Hydrogen_Bond_Donors"] = Lipinski.NumHDonors(molecule)
    properties["Hydrogen_Bond_Acceptors"] = Lipinski.NumHAcceptors(molecule)
    properties["Rotatable_Bonds"] = Lipinski.NumRotatableBonds(molecule)
    properties["Chiral_Centers"] = Chem.FindMolChiralCenters(molecule)

    return properties

In [58]:
df_sample['status'] = df_sample['labels'].map({1: 'Approved', 0 :'Not Approved'})

<ipython-input-58-a2d5da308825>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['status'] = df_sample['labels'].map({1: 'Approved', 0 :'Not Approved'})


In [59]:
df_sample["rdkit_info"] = df_sample["SMILES"].apply(lambda x: get_molecule_properties(x))

<ipython-input-59-54682dfed210>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["rdkit_info"] = df_sample["SMILES"].apply(lambda x: get_molecule_properties(x))


In [60]:
sys_prompt4 ="""
You are a **chemist specializing in drug discovery and molecular modeling**, tasked with analyzing chemical compounds for drug-likeness and viability. Your assessment considers:

1. **Drug-Likeness Rules**:
   - **Lipinski’s Rule of Five** (MW, LogP, H-bond donors/acceptors).
   - **Veber’s Rule** (PSA, rotatable bonds).
   - **Ghose, Egan, and Rule of Three** (lead-like properties).
   - **PAINS & Brenk Filters** (toxic/assay-interfering structures).

2. **ADMET Profiling**:
   - **Absorption** (solubility, permeability).
   - **Distribution** (Vd, plasma protein binding).
   - **Metabolism** (CYP450 interactions, stability).
   - **Excretion** (renal, biliary clearance).
   - **Toxicity** (hepatotoxicity, cardiotoxicity, Ames test).

3. **Information From similar and dissimilar  drugs**


Your goal is to determine the compound’s potential as a drug candidate by integrating these computational and experimental criteria. and predict it will be approved or not
"""

In [122]:
########################################################################################################################
########################################################################################################################
########################################################################################################################


n = 4



########################################################################################################################
########################################################################################################################
########################################################################################################################

In [123]:
def get_first_prompt(list_of_most_approved_info_for_n_similar , list_of_most_nonapproved_info_for_n_similar,rdkit_info , sys_prompt):
    prompt = f"""
    {sys_prompt}

    I have developed a model that can predict the drug likelibility of compound X.
    This model outputs two lists:
    1. The five most similar approved small molecules (with their properties and similarity scores).
    2. The five most similar non-approved small molecules (with their properties and similarity scores).

    Your task is to analyze the likelihood of compound X receiving regulatory approval based on its similarity to known molecules.

    - RDKit Analysis of Compound X:
    {rdkit_info}df["SMILES"]

    Using the provided data:
    - Approved molecules (with similarity scores): {list_of_most_approved_info_for_n_similar}
    - Non-approved molecules (with similarity scores): {list_of_most_nonapproved_info_for_n_similar}
    """
    return prompt

In [124]:
df_sample["1st_prompt"] = df_sample.apply(lambda x : get_first_prompt(x["list_of_most_approved_info_for_n_similar"],x["list_of_most_nonapproved_info_for_n_similar"], x["rdkit_info"], sys_prompt4) ,axis = 1)

<ipython-input-124-e3757a3ddc98>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["1st_prompt"] = df_sample.apply(lambda x : get_first_prompt(x["list_of_most_approved_info_for_n_similar"],x["list_of_most_nonapproved_info_for_n_similar"], x["rdkit_info"], sys_prompt4) ,axis = 1)


In [125]:
df_sample["1st_prompt"][n]

'\n    \nYou are a **chemist specializing in drug discovery and molecular modeling**, tasked with analyzing chemical compounds for drug-likeness and viability. Your assessment considers:\n\n1. **Drug-Likeness Rules**:\n   - **Lipinski’s Rule of Five** (MW, LogP, H-bond donors/acceptors).\n   - **Veber’s Rule** (PSA, rotatable bonds).\n   - **Ghose, Egan, and Rule of Three** (lead-like properties).\n   - **PAINS & Brenk Filters** (toxic/assay-interfering structures).\n\n2. **ADMET Profiling**:\n   - **Absorption** (solubility, permeability).\n   - **Distribution** (Vd, plasma protein binding).\n   - **Metabolism** (CYP450 interactions, stability).\n   - **Excretion** (renal, biliary clearance).\n   - **Toxicity** (hepatotoxicity, cardiotoxicity, Ames test).\n\n3. **Information From similar and dissimilar  drugs**\n\n\nYour goal is to determine the compound’s potential as a drug candidate by integrating these computational and experimental criteria. and predict it will be approved or not

In [126]:
df_sample.loc[n, "think1"] = """
Okay, let me start by looking at the data provided for Compound X. The RDKit analysis gives me some key properties: Molecular Weight is 205.25, LogP is -1.14, TPSA is 89.79, Hydrogen Bond Donors and Acceptors are both 4, Rotatable Bonds are 6, and there are no chiral centers.

First, I need to check the drug-likeness rules. Lipinski's Rule of Five says a molecule should have MW ≤500, which Compound X meets. LogP should be ≤5, and here it's -1.14, so that's good. Hydrogen Bond Donors ≤5 and Acceptors ≤10; both are 4, so that's within limits. So, it passes Lipinski's rules.

Next, Veber's Rule looks at TPSA and Rotatable Bonds. TPSA here is 89.79, which is under the 140 threshold. Rotatable Bonds are 6, which is just at the limit of ≤10, so that's acceptable. So Veber's Rule is satisfied.

Ghose and others: The molecular weight is under 500, LogP is within the -0.4 to 5.6 range, so that's okay. The number of atoms is probably okay since the MW is low. So likely passes here too. PAINS and Brenk filters are for reactive or toxic groups. The approved similar molecules like Dexpanthenol and Miglustat don't have such issues, so maybe Compound X is okay, but without the actual structure, it's hard to say. But given the high similarity to approved drugs, it might not have PAINS issues.

Now looking at the ADMET profiling. Absorption: LogP is low, which could mean good solubility but maybe poor permeability. TPSA is moderate, so maybe moderate absorption. Distribution: With a LogP of -1.14, it might have low plasma protein binding, leading to a higher volume of distribution. Metabolism: The similar approved drugs like Dexpanthenol are metabolized to active forms, but if Compound X is similar, maybe it's metabolized similarly. Excretion: If it's water-soluble, renal clearance might be primary. Toxicity: The approved similar drugs have low toxicity (e.g., Dexpanthenol's LD50 is high), so maybe Compound X is also low toxicity.

Looking at the approved molecules: The top similar ones are Dexpanthenol (0.958), Miglustat (0.888), Glucosamine (0.886), and others. Dexpanthenol is a B vitamin derivative used for skin and intestinal issues, Miglustat is for Gaucher's, Glucosamine for joints. These are approved with good safety profiles. Their properties are similar to Compound X, which suggests that if Compound X is structurally similar, it might have similar ADMET and efficacy.

Non-approved molecules: Duvoglustat (investigational), Migalastat (approved for Fabry), Arbutin (approved for skin), Indole-3-carbinol (investigational). Migalastat is approved but others are investigational. The high similarity to approved drugs like Migalastat and Arbutin (which is approved) might indicate that even though some are non-approved, Compound X's similarity to approved ones is a good sign.

Possible red flags: The high number of rotatable bonds (6) might affect bioavailability, but since similar approved drugs also have similar features, maybe it's acceptable. The TPSA is moderate, which could affect permeability, but again, similar approved drugs have comparable values.

Considering all factors: Compound X meets drug-likeness rules, has ADMET profiles similar to approved drugs, and its closest analogs are approved with favorable toxicity and efficacy. The main concern would be if there's any structural alert not mentioned, but given the data, it's likely to have a good chance of approval.


"""

In [127]:
df_sample.loc[n, "answer1"] = """
Analysis of Compound X for Regulatory Approval Likelihood

1. Drug-Likeness Assessment
Lipinski’s Rule of Five

Molecular Weight (205.25): Passes (≤500).

LogP (-1.14): Passes (≤5).

H-bond Donors (4) / Acceptors (4): Passes (≤5/10).

Conclusion: No violations; excellent oral bioavailability potential.

Veber’s Rule

TPSA (89.79): Passes (≤140 Å²).

Rotatable Bonds (6): Passes (≤10).

Conclusion: Likely good oral absorption.

Ghose/PAINS/Brenk Filters

No structural alerts (e.g., reactive groups) inferred from similarity to approved drugs (e.g., Dexpanthenol, Miglustat).

2. ADMET Profiling
Absorption:

Low LogP (-1.14) suggests high solubility but potential permeability challenges.

Moderate TPSA (89.79) may limit passive diffusion but aligns with approved analogs (e.g., Dexpanthenol: TPSA ~90).

Distribution:

Likely low plasma protein binding (due to low LogP), favoring tissue distribution.

Metabolism:

Similar to Dexpanthenol (converted to pantothenic acid) or Miglustat (minimal hepatic metabolism). Low CYP450 interaction risk.

Excretion:

Predicted renal clearance due to hydrophilicity.

Toxicity:

High LD50 values in analogs (e.g., Dexpanthenol: Mouse LD50 = 15 g/kg oral) suggest low acute toxicity.

No Ames test data, but structural similarity to non-mutagens reduces concern.

3. Similarity to Approved/Non-Approved Molecules
Approved Analogs (High Similarity)
Dexpanthenol (0.958): Approved for epithelial healing. Shares low LogP, moderate TPSA, and metabolism profile.

Miglustat (0.888): Approved for Gaucher’s disease. Matches MW, rotatable bonds, and low toxicity.

Glucosamine (0.886): Over-the-counter joint health. Aligns with solubility and safety trends.

Non-Approved Analogs
Migalastat (0.866): Approved for Fabry disease (accelerated pathway). Suggests niche therapeutic potential.

Arbutin (0.865): Approved for hyperpigmentation. Supports safety in topical applications.

4. Key Strengths
Meets all major drug-likeness rules.

Structural and ADMET similarity to safe, approved drugs.

Low toxicity risk based on analogs.

5. Potential Risks
Permeability: Low LogP may limit passive absorption (needs formulation optimization).

Rotatable Bonds (6): Could affect bioavailability but aligns with approved drugs (e.g., Miglustat: 6 rotatable bonds).

Conclusion: Likely Approval
Compound X demonstrates strong alignment with approved drug profiles, favorable ADMET properties, and no critical structural alerts. Its similarity to Dexpanthenol and Miglustat—both with established safety/efficacy—supports regulatory approval, particularly for indications like epithelial repair or metabolic disorders. Mitigate permeability concerns via formulation (e.g., prodrug strategy) and validate in preclinical models.

Recommendation: Proceed to IND-enabling studies.
"""

In [128]:
def get_second_prompt(label):
    return f"""But wait the true answer is {label}. think step by step and tell why"""
## reevaluation
## on the second thought
df_sample["2nd_prompt"] = df_sample.apply(lambda x : get_second_prompt(x["status"]) ,axis = 1)

<ipython-input-128-31744e05ac3b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["2nd_prompt"] = df_sample.apply(lambda x : get_second_prompt(x["status"]) ,axis = 1)


In [129]:
df_sample["2nd_prompt"][n]

'But wait the true answer is Not Approved. think step by step and tell why'

In [130]:
df_sample.loc[n , "think2"]= """
Okay, let's try to figure out why the compound X wasn't approved even though the initial analysis suggested it might be. First, I need to recall the data provided. The user mentioned that the model listed similar approved and non-approved molecules. The approved ones like Dexpanthenol, Miglustat, Glucosamine, and Nadolol had high similarity scores. The non-approved ones included Duvoglustat, Migalastat, Arbutin, and Indole-3-carbinol.

The initial analysis looked at drug-likeness rules (Lipinski, Veber) and ADMET properties, and everything seemed okay. Molecular weight, LogP, TPSA, rotatable bonds—all within acceptable ranges. Toxicity data from similar approved drugs suggested low acute toxicity. So why the rejection?

Hmm, maybe the answer lies in the specifics of the non-approved molecules. Let me check those again. Migalastat is mentioned as approved but under accelerated pathways, which sometimes means there's conditional approval pending further studies. Duvoglustat is investigational, so maybe it has issues. Arbutin is approved but for topical use, which is different from systemic administration. Indole-3-carbinol is investigational, possibly with unresolved safety issues.

Wait, the user said the true answer is "Not Approved." So what's missing here? Maybe the ADMET profile isn't as good as thought. Let's re-examine the ADMET part. The compound has low LogP (-1.14), which is quite hydrophilic. High solubility is good, but too low LogP can mean poor membrane permeability, making oral bioavailability challenging. Even if similar approved drugs have low LogP, maybe this compound's combination of properties leads to poor absorption despite passing the rules.

Another angle: the similar non-approved drugs. If the compound is structurally closer to non-approved ones with issues, that's a red flag. For example, Migalastat's approval is under accelerated pathways with ongoing trials. If compound X's mechanism or metabolism mirrors a drug that later showed toxicity in confirmatory trials, that could be a problem.

Also, looking at the approved analogs: Dexpanthenol is used topically or as a supplement. Miglustat is for a rare disease (Gaucher) with specific indications. If compound X is intended for a broader use, the safety profile might not hold up. The high similarity to Dexpanthenol might not translate to efficacy in a different therapeutic area.

Toxicity data: The LD50 values from Dexpanthenol are high, but maybe compound X has off-target effects not captured in acute toxicity. Chronic toxicity or specific organ toxicity (like hepatotoxicity) could be a concern. The initial analysis didn't mention CYP interactions or detailed metabolism beyond what's in similar drugs. If compound X has unique metabolites that are toxic, that's a risk.

Pharmacodynamics and mechanism of action: If the approved analogs work through well-understood mechanisms but compound X's MoA is novel or less understood, regulators might be cautious. The non-approved analogs might have mechanisms that failed in clinical trials, and similarity to those could be a negative.

Another point: The presence of 4 hydrogen bond donors and acceptors. While within Lipinski's limits, this might lead to high polarity, affecting passive diffusion. Even if TPSA is under 140, the specific arrangement could hinder absorption.

Also, the rotatable bonds (6) are at the higher end of Veber's criteria. This might increase molecular flexibility, potentially leading to poor bioavailability or rapid metabolism. Approved drugs with similar counts might have optimized structures that compound X lacks.

Looking at the non-approved list, Arbutin is approved but for topical use. If compound X is intended for systemic use, the safety profile from topical applications doesn't guarantee safety orally or intravenously. Similarly, Indole-3-carbinol is investigational, possibly due to insufficient efficacy or safety concerns.

Regulatory agencies also look at therapeutic need. If compound X is similar to existing drugs but doesn't offer significant advantages (e.g., better efficacy, fewer side effects), approval might be denied. The market might already be saturated with similar molecules.

Lastly, maybe the model's similarity scores aren't the whole story. Structural similarity doesn't always translate to similar pharmacokinetics or dynamics. Subtle differences in the structure could lead to unexpected issues in clinical trials, like drug interactions or immune responses not seen in analogs.

Putting it all together, despite passing initial drug-likeness and ADMET checks, compound X's high similarity to drugs with limited use (topical, rare diseases) or non-approved molecules, combined with potential bioavailability issues and lack of therapeutic innovation, likely led to its rejection.

"""

In [131]:
df_sample.loc[n , "answer2"]= """
1. Critical Drug-Likeness and ADMET Red Flags
Lipinski/VEBER Rule Limitations
Low LogP (-1.14): Extremely hydrophilic, suggesting poor membrane permeability despite good solubility. While Lipinski’s rules are met, passive absorption for systemic delivery may fail (e.g., analogs like Dexpanthenol are used topically or in niche indications, not orally).

High Rotatable Bonds (6): At the upper limit of Veber’s criteria, increasing metabolic instability and reducing bioavailability compared to optimized drugs like Miglustat.

ADMET Weaknesses
Absorption: Low LogP and moderate TPSA (89.79) indicate poor oral bioavailability. Approved analogs (e.g., Dexpanthenol) bypass this via non-oral routes (injection/topical).

Toxicity Gaps:

No Ames test data for mutagenicity. Structural similarity to non-mutagens is not definitive.

Chronic toxicity risks (e.g., hepatotoxicity) are unaddressed. Non-approved analogs like Duvoglustat/Indole-3-carbinol highlight unresolved safety concerns.

2. Structural Similarity to High-Risk Analogs
Non-Approved Molecules
Migalastat (0.866 similarity): Approved under accelerated pathways with ongoing confirmatory trials. Compound X’s similarity raises questions about long-term safety validation.

Arbutin (0.865 similarity): Approved only for topical use. Systemic administration of Compound X may inherit unstudied risks (e.g., hydroquinone release).

Duvoglustat/Indole-3-carbinol: Investigational status implies efficacy/safety failures in trials (e.g., anti-HIV activity abandoned due to toxicity).

Approved Molecules with Niche Use
Dexpanthenol: Topical/epithelial use only. Systemic efficacy unproven.

Miglustat: Targets rare diseases (Gaucher/Niemann-Pick). Broad indications for Compound X may require stricter safety thresholds.

3. Lack of Therapeutic Differentiation
Mechanistic Redundancy: Overlaps with existing drugs (e.g., Glucosamine for joint health) without novel targets or improved efficacy.

No Market Need: Similar to Nadolol (beta-blocker), which is no longer first-line due to newer selective agents.

4. Hidden Metabolic/Toxicological Risks
Metabolism: Unlike Miglustat (minimal hepatic metabolism), Compound X’s undefined metabolic pathway risks toxic metabolite formation.

CYP450 Interactions: No data on enzyme inhibition/induction, critical for chronic use.

Conclusion: Key Reasons for Rejection
Poor Oral Bioavailability: Hydrophilicity and rotatable bonds limit systemic absorption.

Structural Links to Non-Approved/High-Risk Drugs: Migalastat’s conditional approval and Arbutin’s topical-only use signal unaddressed risks.

Lack of Therapeutic Innovation: Redundant mechanism without safety/efficacy advantages.

Incomplete Toxicity Profile: Missing mutagenicity and chronic toxicity data.

Final Verdict: Regulatory agencies prioritize risk-benefit balance. Compound X’s similarity to niche-use drugs, unresolved ADMET gaps, and lack of differentiation outweigh its drug-like properties. Not Approved.

Recommendation: Redesign for improved permeability (e.g., prodrug) or pursue topical indications with rigorous chronic toxicity studies."""

In [132]:
## add posible routs(oral/topical):

def get_third_prompt(label):
    return  f"""
    write your thinking in the following format
    {{"status(Approved/NonApproved)": {label},
    "score(0=notapproved to 10=approved): "
    "reason_to_be_approved":
    "reason_to_be_notapproved"   }}
    """
df_sample["3rd_prompt"] = df_sample.apply(lambda x : get_third_prompt(x["status"]) ,axis = 1)

<ipython-input-132-b21bfb3cac8b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["3rd_prompt"] = df_sample.apply(lambda x : get_third_prompt(x["status"]) ,axis = 1)


In [133]:
df_sample["3rd_prompt"][n]

'\n    write your thinking in the following format\n    {"status(Approved/NonApproved)": Not Approved,\n    "score(0=notapproved to 10=approved): "\n    "reason_to_be_approved":\n    "reason_to_be_notapproved"   }\n    '

In [134]:
df_sample.loc[n , "think3"]= """
Alright, the user wants me to format the previous analysis into a specific JSON structure. Let me start by recalling the conclusion from the last response: Compound X was deemed Not Approved.

First, I need to assign a score. The original analysis mentioned strengths like meeting drug-likeness rules and similarity to approved drugs, but significant weaknesses in bioavailability and toxicity. Considering the balance, a score around 3.5/10 seems appropriate—low but not the absolute lowest.

For "reason_to_be_approved", I should list the positives: adherence to Lipinski's and Veber's rules, similarity to safe approved drugs like Dexpanthenol, and low acute toxicity. These are valid points that could support approval under certain conditions.

Now, "reason_to_be_notapproved" requires detailing the critical flaws. Poor oral bioavailability due to low LogP and high rotatable bonds is a key issue. Structural similarity to non-approved or topically used drugs like Arbutin and Migalastat (with their own risks) raises red flags. Lack of data on chronic toxicity and CYP interactions, plus no therapeutic innovation compared to existing drugs, are major hurdles. The market need isn't addressed, making approval unlikely.

I need to ensure the JSON syntax is correct, with proper commas and quotes. Also, the score should be a float. Let me double-check each key and value to match the required format.
"""

In [135]:
df_sample.loc[n , "anwer3"] = """
{
    "status(Approved/NonApproved)": "Not Approved",
    "score(0=notapproved to 10=approved)": 3.5,
    "reason_to_be_approved": "Compound X meets key drug-likeness rules (Lipinski/Veber), aligns structurally with safe approved drugs (e.g., Dexpanthenol, Miglustat), and shows low acute toxicity in analogs.",
    "reason_to_be_notapproved": "1. **Poor oral bioavailability**: Extremely low LogP (-1.14) and high rotatable bonds (6) limit systemic absorption. 2. **Structural similarity to non-approved/topical drugs**: Migalastat (accelerated approval with unresolved safety) and Arbutin (topical-only use) suggest unaddressed systemic risks. 3. **Incomplete toxicity profile**: No Ames test or chronic toxicity data. 4. **No therapeutic differentiation**: Overlaps with existing drugs (e.g., Glucosamine) without novel mechanisms. 5. **Metabolic uncertainty**: Undefined CYP450 interactions and potential toxic metabolites."
}
"""

In [138]:
df_sample.to_csv("/content/drive/MyDrive/Papers/Original_Papers/Llm_drug_prediction/Code/reasoning_data_api/for_sft.csv" , index = False)

## training sft

In [5]:
# df_sft = pd.read_csv("/content/drive/MyDrive/Papers/Original_Papers/Llm_drug_prediction/Code/reasoning_data_api/for_sft.csv" )

In [6]:
# def get_sft_prompt(input , thinking , answer):
#     return f"""<｜begin▁of▁sentence｜><｜User｜>{input}<think>{thinking}</think><｜Assistant｜>{answer}<｜end▁of▁sentence｜><｜Assistant｜>"""


# df_sft["sft_input"] = df_sft.apply(lambda row : get_sft_prompt(row["1st_prompt"] ,f"""{row["think1"]} \n{row["answer1"]} \n but wait lets analysis again \n{row["think2"]}\n {row["answer2"]}"""  , row["anwer3"]) , axis = 1)

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,TrainingArguments

# from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# import torch

# tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,  # Actually 4-bit, but commonly referred to as "8-bit" quantization
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True,
# )


# model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-14B", # this is not working I shoud replace it with 7b
#                                              quantization_config=bnb_config,
#                                              device_map="auto",  # Automatically places layers on available devices
#                                              torch_dtype=torch.bfloat16,
#                                              ) #"manycore-research/SpatialLM-Llama-1B""deepseek-ai/DeepSeek-R1"

# model = prepare_model_for_kbit_training(model)

# peft_config = LoraConfig(
#     r=16,  # Rank of the low-rank matrices
#     lora_alpha=32,  # Scaling factor
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "down_proj" , "up_proj" , "gate_proj"],  # Modules to apply LoRA to
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

# # 6. Wrap model with LoRA
# model = get_peft_model(model, peft_config)


In [9]:
# [len(tokenizer.tokenize(i)) for i in df_sft["sft_input"]]
# df_sft_sft = df_sft[["sft_input"]]
# df_sft_sft.to_csv("/content/drive/MyDrive/Papers/Original_Papers/Llm_drug_prediction/Code/reasoning_data_api/just_for_sft.csv", index = False)

[10556, 4968, 7825, 11845, 11390]

In [5]:
# training_args = SFTConfig(
#     output_dir="/tmp",                  # Directory for output files
#     num_train_epochs=5,                # Number of training epochs
#     per_device_train_batch_size=2,     # Batch size per device
#     gradient_accumulation_steps=16,     # Number of updates steps to accumulate
#     learning_rate=2e-4,                # Learning rate
#     weight_decay=0.01,                 # Weight decay
#     logging_dir="./logs",              # Directory for logs
#     logging_steps=1,                  # Log every X steps
#     save_steps=500,                    # Save checkpoint every X steps
#     save_total_limit=2,                # Max number of checkpoints to keep
#     # evaluation_strategy="steps",       # Evaluate every X steps
#     # eval_steps=100,                    # Evaluation steps
#     fp16=True,                         # Use mixed precision training
#     warmup_steps=500,                  # Number of warmup steps
#     optim="paged_adamw_8bit",
#     lr_scheduler_type="cosine",
#     report_to = "none"
# )

# trainer = SFTTrainer(
#     model = model ,
#     train_dataset=dset2train,
#     args=training_args,
# )
# trainer.train()

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [4]:
df_sft_sft = pd.read_csv("/content/drive/MyDrive/Papers/Original_Papers/Llm_drug_prediction/Code/reasoning_data_api/just_for_sft.csv")

df_sft_sft_100 = pd.concat(100*[df_sft_sft], axis=0, ignore_index=True)

dset = Dataset.from_pandas(df_sft_sft_100)

def get_tokenize(dset):
    return tokenizer(dset["sft_input"] , padding="max_length" , truncation=True  )


dset2train = dset.map(get_tokenize)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from unsloth import FastLanguageModel
import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B",
    # max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args,
)

In [ ]:
trainer_stats = trainer.train()